In [ ]:
from datetime import datetime, timedelta
import heapq
import itertools

# User input collection
def user_input():
    weekday_hours = int(input("Enter weekday study hours per day: "))
    weekend_hours = int(input("Enter weekend study hours per day: "))
    preferred_slots = input("Enter preferred time slots (e.g., morning/evening): ").split("/")
    num_subjects = int(input("How many subjects/topics do you want to add? "))

    subjects_data = {}
    deadlines = {}
    min_study_times = {}

    for i in range(num_subjects):
        subject = input(f"Enter subject/topic {i+1}: ")
        importance = input(f"Importance level for {subject} (High/Medium/Low): ")
        date = input(f"Enter deadline for {subject} (YYYY-MM-DD): ")
        deadline = datetime.strptime(date, '%Y-%m-%d').date()
        min_time = int(input(f"Minimum study time for {subject} (in hours): "))

        subjects_data[subject] = {'importance': importance}
        deadlines[subject] = deadline
        min_study_times[subject] = min_time

    max_hours = int(input("Maximum study hours per day you can handle? "))

    return {
        'study_time': {
            'weekday_hours': weekday_hours,
            'weekend_hours': weekend_hours,
            'days_available': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
            'preferred_slots': preferred_slots,
            'min_study_times': min_study_times
        },
        'subjects': subjects_data,
        'deadlines': deadlines,
        'system_constraints': {
            'max_study_hours_per_day': max_hours
        }
    }

# Goal check
def is_goal_reached(plan, subjects, min_study_times):
    for subj in subjects:
        required_time = min_study_times.get(subj, 3)
        if plan.get(subj, 0) < required_time:
            return False
    return True

# Generate possible next study plans (multi-day version)
def generate_possible_plans(current_plan, subjects, min_study_times, deadlines):
    possible_plans = []
    today = datetime.now().date()

    # Sort subjects by nearest deadline
    sorted_subjects = sorted(subjects, key=lambda subj: (deadlines[subj] - today).days)

    for subj in sorted_subjects:
        new_plan = current_plan.copy()
        if new_plan.get(subj, 0) < min_study_times[subj]:
            addition = min(2, min_study_times[subj] - new_plan[subj])
            new_plan[subj] = new_plan.get(subj, 0) + addition
            possible_plans.append(new_plan)
    return possible_plans

# Cost function
def calculate_cost(plan, subjects, min_study_times):
    cost = 0
    for subj in subjects:
        if plan.get(subj, 0) < min_study_times[subj]:
            cost += 1
        else:
            cost += 0.5
    return cost

# Heuristic function
def calculate_heuristic(plan, subjects, min_study_times, deadlines):
    heuristic = 0
    current_date = datetime.now().date()
    for subj in subjects:
        days_until_deadline = (deadlines[subj] - current_date).days
        if plan.get(subj, 0) < min_study_times[subj]:
            heuristic += max(1, 10 - days_until_deadline // 2)
        else:
            heuristic += max(1, days_until_deadline // 5)
    return heuristic

# A* Search Algorithm
def a_star_search(subjects, deadlines, min_study_times):
    counter = itertools.count()
    initial_plan = {subj: 0 for subj in subjects}
    open_list = []
    heapq.heappush(open_list, (0, 0, next(counter), initial_plan))
    closed_list = []

    while open_list:
        current_cost, current_heuristic, _, current_plan = heapq.heappop(open_list)

        if is_goal_reached(current_plan, subjects, min_study_times):
            return current_plan

        for next_plan in generate_possible_plans(current_plan, subjects, min_study_times, deadlines):
            if next_plan not in closed_list:
                next_cost = current_cost + calculate_cost(next_plan, subjects, min_study_times)
                next_heuristic = calculate_heuristic(next_plan, subjects, min_study_times, deadlines)
                total_cost = next_cost + next_heuristic
                heapq.heappush(open_list, (total_cost, next_heuristic, next(counter), next_plan))
                closed_list.append(next_plan)

    return None

# Intelligent Agent
class StudyPlannerAgent:
    def __init__(self, user_profile):
        self.user_profile = user_profile
        self.study_plan = {}

    def perceive_environment(self):
        self.time_data = self.user_profile['study_time']
        self.subjects = self.user_profile['subjects']
        self.deadlines = self.user_profile['deadlines']
        self.constraints = self.user_profile['system_constraints']

    def plan_schedule(self):
        print("\nPlanning optimal study schedule\n")
        self.study_plan = a_star_search(
            self.subjects,
            self.deadlines,
            self.time_data['min_study_times']
        )

    def act(self):
        return self.study_plan

    def run(self):
        self.perceive_environment()
        self.plan_schedule()
        return self.act()

def distribute_study_plan(final_plan, user_profile):
    from collections import deque

    weekday_hours = user_profile['study_time']['weekday_hours']
    weekend_hours = user_profile['study_time']['weekend_hours']
    preferred_slots = user_profile['study_time']['preferred_slots']
    today = datetime.now().date()

    full_schedule = {}
    day_counter = 0
    subjects_remaining = final_plan.copy()
    subjects_queue = deque([subj for subj in subjects_remaining if subjects_remaining[subj] > 0])

    while any(hours > 0 for hours in subjects_remaining.values()):
        current_day = today + timedelta(days=day_counter)
        day_name = current_day.strftime("%A")
        daily_limit = weekend_hours if day_name in ["Saturday", "Sunday"] else weekday_hours
        today_schedule = {}
        available_hours = daily_limit

        # Make a fresh queue each day to ensure fairness
        subjects_queue = deque([subj for subj in subjects_remaining if subjects_remaining[subj] > 0])

        while available_hours > 0 and subjects_queue:
            subj = subjects_queue.popleft()
            if subjects_remaining[subj] == 0:
                continue
            # Allocate at least 1 hour or the remaining available hours
            study_hours = min(1, available_hours, subjects_remaining[subj])
            # If more hours are available, try to allocate more evenly
            # You can change this to allocate more per subject if needed
            today_schedule[subj] = today_schedule.get(subj, 0) + study_hours
            subjects_remaining[subj] -= study_hours
            available_hours -= study_hours
            # If this subject still needs hours, put it back in the queue
            if subjects_remaining[subj] > 0:
                subjects_queue.append(subj)

        full_schedule[current_day.strftime("%Y-%m-%d")] = {
            'tasks': today_schedule,
            'slot': preferred_slots[day_counter % len(preferred_slots)]
        }
        day_counter += 1
    return full_schedule

In [ ]:
def collect_performance_data(subjects):
    performance_data = {}
    print("\nEnter your latest quiz/test scores to help improve your study plan:")
    for subject in subjects:
        while True:
            try:
                score = float(input(f"Score for {subject} (0-100): "))
                if 0 <= score <= 100:
                    performance_data[subject] = score
                    break
                else:
                    print("Score must be between 0 and 100. Please try again.")
            except ValueError:
                print("Please enter a valid number.")
    return performance_data

class StudyTimePredictor:
    def __init__(self):
        self.data = {
            'study_hours': [],
            'prior_scores': [],
            'new_scores': []
        }
        self.model_trained = False
        self.coefficients = {'intercept': 0, 'hours': 0.5, 'prior': 0.5}

    def add_data_point(self, subject, study_hours, prior_score, new_score):
        self.data['study_hours'].append(study_hours)
        self.data['prior_scores'].append(prior_score)
        self.data['new_scores'].append(new_score)

    def train_model(self):
        if len(self.data['study_hours']) >= 5:
            try:
                x_hours = self.data['study_hours']
                x_prior = self.data['prior_scores']
                y = self.data['new_scores']

                mean_hours = sum(x_hours) / len(x_hours)
                mean_prior = sum(x_prior) / len(x_prior)
                mean_y = sum(y) / len(y)

                numerator_hours = sum((x - mean_hours) * (y[i] - mean_y)
                                 for i, (x, y) in enumerate(zip(x_hours, y)))
                denominator_hours = sum((x - mean_hours) ** 2 for x in x_hours)

                numerator_prior = sum((x - mean_prior) * (y[i] - mean_y)
                                 for i, (x, y) in enumerate(zip(x_prior, y)))
                denominator_prior = sum((x - mean_prior) ** 2 for x in x_prior)

                if denominator_hours and denominator_prior:
                    self.coefficients['hours'] = numerator_hours / denominator_hours
                    self.coefficients['prior'] = numerator_prior / denominator_prior
                    self.coefficients['intercept'] = mean_y - (self.coefficients['hours'] * mean_hours) - \
                                                    (self.coefficients['prior'] * mean_prior)
                    self.model_trained = True
                    return True
            except Exception as e:
                print(f"Error training model: {e}")
                return False
        return False

    def predict_hours_needed(self, prior_score, target_score=85):
        if not self.model_trained:
            if prior_score < 60:
                return 4
            elif prior_score < 75:
                return 3
            else:
                return 2

        if self.coefficients['hours'] == 0:
            return 3

        hours = (target_score - self.coefficients['intercept'] -
                (self.coefficients['prior'] * prior_score)) / self.coefficients['hours']

        hours = max(1, min(6, hours))
        return round(hours)

def adapt_study_plan(user_profile, performance_data, predictor):
    min_study_times = user_profile['study_time']['min_study_times']

    for subject, score in performance_data.items():
        if 'performance_history' not in user_profile['subjects'][subject]:
            user_profile['subjects'][subject]['performance_history'] = []

        user_profile['subjects'][subject]['performance_history'].append(score)

        prior_scores = user_profile['subjects'][subject].get('performance_history', [])
        prior_score = prior_scores[-2] if len(prior_scores) > 1 else 70

        current_time = min_study_times.get(subject, 3)

        predictor.add_data_point(subject, current_time, prior_score, score)

        if predictor.model_trained:
            recommended_hours = predictor.predict_hours_needed(score)
        else:
            if score < 70:
                recommended_hours = current_time + 1
            elif score > 85 and current_time > 1:
                recommended_hours = current_time - 0.5
            else:
                recommended_hours = current_time

        min_study_times[subject] = max(1, min(6, recommended_hours))

    predictor.train_model()

    return user_profile

def calculate_evaluation_metrics(user_profile, performance_data, study_schedule):
    metrics = {
        'deadline_compliance': 0,
        'performance_improvement': 0,
        'time_management': 0,
        'subject_coverage': 0
    }

    total_subjects = len(user_profile['deadlines'])
    if total_subjects > 0:
        for subject, deadline in user_profile['deadlines'].items():
            subject_complete = False
            for date, day_schedule in study_schedule.items():
                schedule_date = datetime.strptime(date, '%Y-%m-%d').date()
                if schedule_date > deadline:
                    break
                if subject in day_schedule['tasks']:
                    subject_complete = True

            if subject_complete:
                metrics['deadline_compliance'] += 1

        metrics['deadline_compliance'] = (metrics['deadline_compliance'] / total_subjects) * 100

    improvements = 0
    subjects_with_history = 0
    for subject, data in user_profile['subjects'].items():
        history = data.get('performance_history', [])
        if len(history) >= 2:
            if history[-1] > history[-2]:
                improvements += 1
            subjects_with_history += 1

    if subjects_with_history > 0:
        metrics['performance_improvement'] = (improvements / subjects_with_history) * 100

    daily_hours_limit = user_profile['system_constraints']['max_study_hours_per_day']
    total_possible_hours = 0
    total_scheduled_hours = 0

    for date, day_schedule in study_schedule.items():
        schedule_date = datetime.strptime(date, '%Y-%m-%d').date()
        weekday = schedule_date.weekday()

        if weekday < 5:
            day_limit = user_profile['study_time']['weekday_hours']
        else:
            day_limit = user_profile['study_time']['weekend_hours']

        day_limit = min(day_limit, daily_hours_limit)
        total_possible_hours += day_limit

        day_total = sum(day_schedule['tasks'].values())
        total_scheduled_hours += day_total

    if total_possible_hours > 0:
        metrics['time_management'] = (total_scheduled_hours / total_possible_hours) * 100

    subjects_covered = 0
    for subject, min_time in user_profile['study_time']['min_study_times'].items():
        total_hours = 0
        for date, day_schedule in study_schedule.items():
            if subject in day_schedule['tasks']:
                total_hours += day_schedule['tasks'][subject]

        if total_hours >= min_time:
            subjects_covered += 1

    if total_subjects > 0:
        metrics['subject_coverage'] = (subjects_covered / total_subjects) * 100

    return metrics

def display_evaluation_metrics(metrics):
    print("\nStudy Plan Evaluation Metrics:")
    print(f"  Deadline Compliance: {metrics['deadline_compliance']:.1f}%")
    print(f"  Performance Improvement: {metrics['performance_improvement']:.1f}%")
    print(f"  Time Management Efficiency: {metrics['time_management']:.1f}%")
    print(f"  Subject Coverage: {metrics['subject_coverage']:.1f}%")

    overall = (metrics['deadline_compliance'] +
              metrics['performance_improvement'] +
              metrics['time_management'] +
              metrics['subject_coverage']) / 4

    print(f"  Overall Plan Effectiveness: {overall:.1f}%")

    print("\n💡 Suggestions for Improvement:")
    if metrics['deadline_compliance'] < 80:
        print("  - Consider starting earlier on subjects with approaching deadlines")
    if metrics['performance_improvement'] < 70:
        print("  - Review your study techniques; focus on understanding rather than memorization")
    if metrics['time_management'] < 80:
        print("  - You're not using all your available study time efficiently")
    if metrics['subject_coverage'] < 90:
        print("  - Some subjects aren't receiving their minimum required study time")

def run_study_planner_loop(weeks=4):
    user_profile = user_input()
    predictor = StudyTimePredictor()

    for subject in user_profile['subjects']:
        user_profile['subjects'][subject]['performance_history'] = []

    all_schedules = {}

    for week in range(weeks):
        print(f"\n{'='*20} Week {week+1} Study Plan {'='*20}")

        agent = StudyPlannerAgent(user_profile)
        final_plan = agent.run()

        if not final_plan:
            print("Could not find a feasible plan within given constraints!")
            break

        print("\nRecommended Study Plan (hours per subject):")
        for subject, hours in final_plan.items():
            print(f"{subject}: {hours} hours")

        print("\n📅 Detailed Day-by-Day Study Plan:")
        week_schedule = distribute_study_plan(final_plan, user_profile)
        for date, schedule in week_schedule.items():
            print(f"Date: {date} ({schedule['slot'].capitalize()} session)")
            for subj, hrs in schedule['tasks'].items():
                print(f"  {subj}: {hrs} hour(s)")
            print()

        all_schedules.update(week_schedule)

        if week < weeks - 1:
            print("\n📝 Let's collect feedback on your progress...")
            performance_data = collect_performance_data(user_profile['subjects'])
            user_profile = adapt_study_plan(user_profile, performance_data, predictor)

            print("\n✅ Your study plan has been updated based on your performance!")
            print("Here are your updated minimum study hours:")
            for subject, hours in user_profile['study_time']['min_study_times'].items():
                print(f"  {subject}: {hours} hours")

    if user_profile['subjects'] and any('performance_history' in data for data in user_profile['subjects'].values()):
        latest_performance = {
            subject: data['performance_history'][-1]
            for subject, data in user_profile['subjects'].items()
            if 'performance_history' in data and data['performance_history']
        }

        metrics = calculate_evaluation_metrics(user_profile, latest_performance, all_schedules)
        display_evaluation_metrics(metrics)

if __name__ == '__main__':
    print("AI Study Planner")
    print("This planner will create a personalized study schedule and adapt over time.")
    print("It will run for 4 weeks, collecting feedback each week to improve your plan.")
    try:
        run_study_planner_loop(weeks=2)
    except KeyboardInterrupt:
        print("\n\nStudy planner exited.")
    except Exception as e:
        print(f"\n\nAn error occurred: {e}")

    print("\nThank you for using the AI Study Planner!")


AI Study Planner
This planner will create a personalized study schedule and adapt over time.
It will run for 4 weeks, collecting feedback each week to improve your plan.
Enter weekday study hours per day: 5
Enter weekend study hours per day: 8
Enter preferred time slots (e.g., morning/evening): morning
How many subjects/topics do you want to add? 3
Enter subject/topic 1: ai
Importance level for ai (High/Medium/Low): high
Enter deadline for ai (YYYY-MM-DD): 2025-05-10
Minimum study time for ai (in hours): 15
Enter subject/topic 2: daa
Importance level for daa (High/Medium/Low): medium
Enter deadline for daa (YYYY-MM-DD): 2025-05-15
Minimum study time for daa (in hours): 10
Enter subject/topic 3: cc
Importance level for cc (High/Medium/Low): low
Enter deadline for cc (YYYY-MM-DD): 2025-05-05
Minimum study time for cc (in hours): 6
Maximum study hours per day you can handle? 6

==================== Week 1 Study Plan ====================

Planning optimal study schedule


Recommended Study